Copyright 2021 National Technology & Engineering Solutions<br>
of Sandia, LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS,<br>
the U.S. Government retains certain rights in this software.<br>
<br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br>
You may obtain a copy of the License at<br>
<br>
   http://www.apache.org/licenses/LICENSE-2.0<br>
<br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.

In [1]:
import logging

In [2]:
import numpy

In [3]:
import cicada.additive
import cicada.communicator

In [4]:
logging.basicConfig(level=logging.INFO)

In this example we'll be generating uniformly random shared secrets then we will clear out any fractional parts of the secrets leaving only the integer numeric components behind. We reveal the operands and the results to demonstrate correctness.

You can see from the code below that we do apply a mod 1 operation when we print the values, this is to assure that the fractional parts of each value can clearly be seen in the before and after. The operands selected uniformly at random from the field can be quite large and the fractional parts would not be able to be seen with numpy's representation of them otherwise. Following the operations we would expect all zeros to be returned given that we are clearing the fractional parts and applying modulus 1.

You may of course feel free to remove the mod 1 operations to see the values without that being applied.

In [12]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    generator = numpy.random.default_rng()
    secret_share = protocol.uniform(shape=(2,2))
    secret = protocol.encoder.decode(protocol.reveal(secret_share))
    log.info(f"Player {communicator.rank} secret: \n{secret%1}")
    floor_share = protocol.floor(secret_share)
    floor = protocol.encoder.decode(protocol.reveal(floor_share))
    log.info(f"Player {communicator.rank} floor: \n{floor%1}")

In [13]:
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:56077 from tcp://127.0.0.1:56077.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:56077 from tcp://127.0.0.1:56078.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:56077 from tcp://127.0.0.1:56079.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 secret: 
[[0.40625  0.703125]
 [0.25     0.828125]]
INFO:root:Player 1 secret: 
[[0.40625  0.703125]
 [0.25     0.828125]]
INFO:root:Player 2 secret: 
[[0.40625  0.703125]
 [0.25     0.828125]]
INFO:root:Player 0 floor: 
[[0. 0.]
 [0. 0.]]
INFO:root:Player 1 floor: 
[[0. 0.]
 [0. 0.]]
INFO:root:Player 2 floor: 
[[0. 0.]
 [0. 0.]]
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 co

[None, None, None]